In [9]:
# Set up and load data
# Includes
import sys
import os
import numpy as np
import json
import os


# Setup paths containing utility
curr_folder = os.getcwd()
sys.path.insert(0, os.path.join(curr_folder,'../app'))

# Utils imports for loading data
from utils import save_data, load_data, exists_datafolder
from utils import load_SQuAD_train, load_SQuAD_dev
from utils import get_foldername
from utils import merge_artfiles


# Load data containing NEP (predictions)
foldername = get_foldername('sq_pp_ner')
arts = load_data('train.json',foldername)
print(arts[1]['title'])
print(arts[1]['paragraphs'][0]['context'])


Frédéric_Chopin
Frédéric François Chopin (/ˈʃoʊpæn/; French pronunciation: ​[fʁe.de.ʁik fʁɑ̃.swa ʃɔ.pɛ̃]; 22 February or 1 March 1810 – 17 October 1849), born Fryderyk Franciszek Chopin,[n 1] was a Polish and French (by citizenship and birth of father) composer and a virtuoso pianist of the Romantic era, who wrote primarily for the solo piano. He gained and has maintained renown worldwide as one of the leading musicians of his era, whose "poetic genius was based on a professional technique that was without equal in his generation." Chopin was born in what was then the Duchy of Warsaw, and grew up in Warsaw, which after 1815 became part of Congress Poland. A child prodigy, he completed his musical education and composed his earlier works in Warsaw before leaving Poland at the age of 20, less than a month before the outbreak of the November 1830 Uprising.


In [11]:
# Load blanks data (ground truth)
foldername = get_foldername('sq_pp_training')
arts3 = load_data('train.json',foldername)
print(arts3[1]['title'])
print(arts3[1]['paragraphs'][0]['context_blanked'])


Frédéric_Chopin
Frédéric François Chopin (/ˈʃoʊpæn/; ______ pronunciation: ​[fʁe.de.ʁik fʁɑ̃.swa ʃɔ.pɛ̃]; 22 February or 1 March ______ – 17 October 1849), born Fryderyk Franciszek Chopin,[n 1] was a ______ and ______ (by citizenship and birth of father) composer and a virtuoso pianist of the ______ era, who wrote primarily for the ______ piano. He gained and has maintained renown worldwide as one of the leading musicians of his era, whose "poetic genius was based on a professional technique that was without equal in his generation." Chopin was born in what was then the ______ of Warsaw, and grew up in Warsaw, which after 1815 became part of Congress Poland. A child prodigy, he completed his musical education and composed his earlier works in ______ before leaving Poland at the age of 20, less than a month before the outbreak of the November 1830 Uprising.


In [14]:
# Merge ground truth blanks with original data to get full dataset
from utils_SQuAD import merge_arts_paragraph_fields
list_of_fields = ['context_blanked','blank_classification']
arts = merge_arts_paragraph_fields(arts,arts3,list_of_fields)

print(arts[1]['title'])
print(arts[1]['paragraphs'][0]['context'])
print(arts[1]['paragraphs'][0]['context_blanked'])

Frédéric_Chopin
Frédéric François Chopin (/ˈʃoʊpæn/; French pronunciation: ​[fʁe.de.ʁik fʁɑ̃.swa ʃɔ.pɛ̃]; 22 February or 1 March 1810 – 17 October 1849), born Fryderyk Franciszek Chopin,[n 1] was a Polish and French (by citizenship and birth of father) composer and a virtuoso pianist of the Romantic era, who wrote primarily for the solo piano. He gained and has maintained renown worldwide as one of the leading musicians of his era, whose "poetic genius was based on a professional technique that was without equal in his generation." Chopin was born in what was then the Duchy of Warsaw, and grew up in Warsaw, which after 1815 became part of Congress Poland. A child prodigy, he completed his musical education and composed his earlier works in Warsaw before leaving Poland at the age of 20, less than a month before the outbreak of the November 1830 Uprising.
Frédéric François Chopin (/ˈʃoʊpæn/; ______ pronunciation: ​[fʁe.de.ʁik fʁɑ̃.swa ʃɔ.pɛ̃]; 22 February or 1 March ______ – 17 October 1

In [15]:
# All articles
Narticles = len(arts)
print ("Narticles = " +  str(Narticles)) 


Narticles = 442


In [17]:
arts[1]['paragraphs'][0]['allenNER']


{'words': 'Frédéric François Chopin ( /ˈʃoʊpæn/ ; French pronunciation : \u200b[fʁe.de.ʁik fʁɑ̃.swa ʃɔ.pɛ̃ ] ; 22 February or 1 March 1810 – 17 October 1849 ) , born Fryderyk Franciszek Chopin,[n 1 ] was a Polish and French ( by citizenship and birth of father ) composer and a virtuoso pianist of the Romantic era , who wrote primarily for the solo piano . He gained and has maintained renown worldwide as one of the leading musicians of his era , whose " poetic genius was based on a professional technique that was without equal in his generation . " Chopin was born in what was then the Duchy of Warsaw , and grew up in Warsaw , which after 1815 became part of Congress Poland . A child prodigy , he completed his musical education and composed his earlier works in Warsaw before leaving Poland at the age of 20 , less than a month before the outbreak of the November 1830 Uprising .',
 'tags': 'B-PER I-PER L-PER O U-PER O U-MISC O O O O O O O O O O O O O O O O O O O O B-PER I-PER L-PER O O O O

In [5]:
art = arts[15:17]
# for a in art:
#     for p in a['paragraphs']:
#         print(p['context'] + '\n') 

In [6]:
#arts[0]['paragraphs'][0]['qas'][0]['answers']


answers = []
all_questions = [qa['question'] for a in art for p in a['paragraphs'] for qa in p['qas'] for a in qa['answers']]
all_answers = [a['text'] for a in art for p in a['paragraphs'] for qa in p['qas'] for a in qa['answers']]
answer_is_verbatim_in_context = [a['text'] in p['context'] for a in art for p in a['paragraphs'] for qa in p['qas'] for a in qa['answers']]

print('Num answers: ' + str(len(all_answers)))
print('Num true bools: ' + str(sum(answer_is_verbatim_in_context)))


Num answers: 4837
Num true bools: 4837


In [30]:


text = [p['context'] for a in art for p in a['paragraphs']]
text = ' '.join(text[:])

text


'In modern molecular biology and genetics, the genome is the genetic material of an organism. It consists of DNA (or RNA in RNA viruses). The genome includes both the genes and the non-coding sequences of the DNA/RNA. The term was created in 1920 by Hans Winkler, professor of botany at the University of Hamburg, Germany. The Oxford Dictionary suggests the name to be a blend of the words gene and chromosome. However, see omics for a more thorough discussion. A few related -ome words already existed—such as biome, rhizome, forming a vocabulary into which genome fits systematically. Some organisms have multiple copies of chromosomes: diploid, triploid, tetraploid and so on. In classical genetics, in a sexually reproducing organism (typically eukarya) the gamete has half the number of chromosomes of the somatic cell and the genome is a full set of chromosomes in a diploid cell. The halving of the genetic material in gametes is accomplished by the segregation of homologous chromosomes durin

In [31]:
# Set up and test AllenNLP
# Tags PER, ORG and LOC denote person names, organizations and locations respectively

from allennlp.predictors import Predictor
predictor = Predictor.from_path("/home/davestanley/src/allennlp/ner-model-2018.12.18.tar.gz")
results = predictor.predict(sentence=text)
# for word, tag in zip(results["words"], results["tags"]):
#     print(f"{word}\t{tag}")


01/25/2019 06:00:41 - INFO - allennlp.models.archival -   loading archive file /home/davestanley/src/allennlp/ner-model-2018.12.18.tar.gz
01/25/2019 06:00:41 - INFO - allennlp.models.archival -   extracting archive file /home/davestanley/src/allennlp/ner-model-2018.12.18.tar.gz to temp dir /tmp/tmpgg4sl46s
01/25/2019 06:00:46 - INFO - allennlp.common.params -   type = default
01/25/2019 06:00:46 - INFO - allennlp.data.vocabulary -   Loading token dictionary from /tmp/tmpgg4sl46s/vocabulary.
01/25/2019 06:00:46 - INFO - allennlp.common.from_params -   instantiating class <class 'allennlp.models.model.Model'> from params {'type': 'crf_tagger', 'encoder': {'bidirectional': True, 'dropout': 0.5, 'hidden_size': 200, 'input_size': 1202, 'num_layers': 2, 'type': 'lstm'}, 'regularizer': [['scalar_parameters', {'alpha': 0.1, 'type': 'l2'}]], 'label_encoding': 'BIOUL', 'include_start_end_transitions': False, 'dropout': 0.5, 'text_field_embedder': {'tokens': {'embedding_dim': 50, 'trainable': Tru

In [34]:
for word, tag in zip(results["words"], results["tags"]):
    print(f"{word}\t{tag}")

In	O
modern	O
molecular	O
biology	O
and	O
genetics	O
,	O
the	O
genome	O
is	O
the	O
genetic	O
material	O
of	O
an	O
organism	O
.	O
It	O
consists	O
of	O
DNA	U-MISC
(	O
or	O
RNA	U-MISC
in	O
RNA	U-MISC
viruses	O
)	O
.	O
The	O
genome	O
includes	O
both	O
the	O
genes	O
and	O
the	O
non	O
-	O
coding	O
sequences	O
of	O
the	O
DNA	U-MISC
/	O
RNA	U-MISC
.	O
The	O
term	O
was	O
created	O
in	O
1920	O
by	O
Hans	B-PER
Winkler	L-PER
,	O
professor	O
of	O
botany	O
at	O
the	O
University	B-ORG
of	I-ORG
Hamburg	L-ORG
,	O
Germany	U-LOC
.	O
The	O
Oxford	B-ORG
Dictionary	L-ORG
suggests	O
the	O
name	O
to	O
be	O
a	O
blend	O
of	O
the	O
words	O
gene	O
and	O
chromosome	O
.	O
However	O
,	O
see	O
omics	O
for	O
a	O
more	O
thorough	O
discussion	O
.	O
A	O
few	O
related	O
-ome	O
words	O
already	O
existed	O
—	O
such	O
as	O
biome	O
,	O
rhizome	O
,	O
forming	O
a	O
vocabulary	O
into	O
which	O
genome	O
fits	O
systematically	O
.	O
Some	O
organisms	O
have	O
multiple	O
copies	O
of	O
chromosomes	O
:	O
diploid	O
,	O
triploid	O
,	O
te

In [35]:
len(text.split())

4018

In [36]:
len(results['words'])

4589